In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from datetime import datetime
import pulp

names = ['VIN', 'plant', 'dealer','time']
df = pd.read_csv('Problem_VehicleShipmentRequirement.csv',names=names,header=0,usecols=[0,1,2,3])
names_vdc = ['plant','capacity','rail']
df_vdc = pd.read_csv('existingvdc.csv',names=names_vdc,header=0,usecols=[0,1,2])
names_locations = ['location', 'dealer','lat','long']
df_locations = pd.read_csv('locations.csv',names=names_locations,header=0,usecols=[0,1,2,3])
locations_lp = pd.read_csv('locations.csv', index_col=['Location'])

In [3]:
df_locations['dealer'].value_counts()
dealers = df_locations[df_locations.dealer == 1]
VDCs = df_locations[df_locations.dealer == 0]

In [4]:
dealers_lats = dealers['lat'].tolist()
dealers_longs = dealers['long'].tolist()
dealers_locations = dealers['location'].tolist()

In [5]:
initialROs = df[df.plant == 'RO']
initialFFs = df[df.plant == 'FF']
initial3As = df[df.plant == '3A']
initialSOs = df[df.plant == 'SO']

print(len(initialROs))
print(len(initialFFs))
print(len(initial3As))
print(len(initialSOs))

311842
171269
237191
328273


In [6]:
import math

def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 3959 # miles

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [64]:
#PuLp doesn't understand floor divide so have to create own



In [59]:
#Create list of VDCs with rails

vdcrails = df_vdc['rail'].tolist()
existingvdcs = df_vdc['plant'].tolist()
print(vdcrails)

vdcswithrails = []

for i in range(len(vdcrails)):
    if vdcrails[i]:
        vdcswithrails.append(existingvdcs[i])
        
print(vdcswithrails)
        

[True, True, False, True, True, False, False, True, True, False, False, True, True, True, True, True, True, False, False, False, False, False, False, True, False, False, False, True, False, True, False, True, True, False, False, True, False, False, False, True, True, True, False, True]
['3A', '3F', '7J', '7M', 'BM', 'CE', 'DO', 'DV', 'DW', 'DZ', 'EC', 'FF', 'NZ', 'QT', 'RO', 'RX', 'SO', 'UL', 'VW', 'WL', 'QW', 'WK']


In [60]:
#Instantiate problem class

model = pulp.LpProblem("IEOR 162 final project", pulp.LpMinimize)

In [61]:
#Construct our decision variable lists

VINs = df['VIN'].tolist()
VDCs = df_vdc['plant'].tolist()
dealers_var = dealers['location'].tolist()

capacities = df_vdc['capacity'].tolist()
dealers_3A = initial3As['dealer'].tolist()
dealers_RO = initialROs['dealer'].tolist()
dealers_SO = initialSOs['dealer'].tolist()
dealers_FF = initialFFs['dealer'].tolist()

newdf_3A = initial3As.groupby('dealer').count()
newdf_RO = initialROs.groupby('dealer').count()
newdf_SO = initialSOs.groupby('dealer').count()
newdf_FF = initialFFs.groupby('dealer').count()

vins_3A = newdf_3A['VIN'].tolist()
vins_RO = newdf_RO['VIN'].tolist()
vins_SO = newdf_SO['VIN'].tolist()
vins_FF = newdf_FF['VIN'].tolist()

curr_index_3A = newdf_3A.index.get_values()
curr_index_RO = newdf_RO.index.get_values()
curr_index_SO = newdf_SO.index.get_values()
curr_index_FF = newdf_FF.index.get_values()

new_index_3A = []
new_index_RO = []
new_index_SO = []
new_index_FF = []

for item in curr_index_3A:
    temp = str(item)
    new_index_3A.append(temp)
for item in curr_index_RO:
    temp = str(item)
    new_index_RO.append(temp)
for item in curr_index_SO:
    temp = str(item)
    new_index_SO.append(temp)
for item in curr_index_FF:
    temp = str(item)
    new_index_FF.append(temp)
    
newdf_3A.index = new_index_3A #Setting grouped dataframe's index as dealer
newdf_RO.index = new_index_RO
newdf_SO.index = new_index_SO
newdf_FF.index = new_index_FF
#print(new_index)
#print(newdf)

combined_index = set(new_index_3A + new_index_SO + new_index_RO + new_index_FF)

print(len(dealers_3A))
print(len(dealers_RO))
print(len(dealers_SO))
print(len(dealers_FF))
print(len(dealers_var))

print(len(new_index_3A))
print(len(new_index_SO))
print(len(new_index_RO))
print(len(new_index_FF))

print(len(combined_index))
print(len(set(combined_index)))
#print(dealers_var[:10])

237191
311842
328273
171269
21611
7289
6280
5935
5827
8131
8131


In [62]:
#Instantiate variable classes

truck_vdc = pulp.LpVariable.dicts("# of cars shipped between vdc i and j by truck", ((str(i), str(j)) for i in VDCs for j in VDCs if i != j), lowBound=0, cat='Integer')
truck_dealer = pulp.LpVariable.dicts("# of cars shipped between vdc i and dealer k by truck", ((str(i), str(k)) for i in VDCs for k in dealers_var), lowBound=0, cat='Integer')
arrive_by_truck = pulp.LpVariable.dicts("# of cars arriving at dealer k from vdc i", ((str(i), str(k)) for i in ['3A', 'RO', 'SO', 'FF'] for k in combined_index), lowBound=0, cat='Integer')

rail_vdc = pulp.LpVariable.dicts("# of cars shipped between vdc i and j by rail", ((str(i), str(j)) for i in vdcswithrails for j in vdcswithrails if i != j), lowBound=0, cat='Integer')
rail_dealer = pulp.LpVariable.dicts("# of cars shipped between vdc i and dealer j by rail", ((str(i), str(k)) for i in vdcswithrails for k in dealers_var), lowBound=0, cat='Integer')

In [76]:
#Construct objective function

for i in VDCs:
    model += (pulp.lpSum([(200 + (4)*distance([locations_lp.loc[str(i), 'Latitude'], locations_lp.loc[str(i), 'Longitude']],[locations_lp.loc[str(j), 'Latitude'], locations_lp.loc[str(j), 'Longitude']]))*((truck_dealer[(str(i), str(j))])*(1/10)) for j in dealers_var]))
    model += (pulp.lpSum([(200 + (4)*distance([locations_lp.loc[str(i), 'Latitude'], locations_lp.loc[str(i), 'Longitude']],[locations_lp.loc[str(j), 'Latitude'], locations_lp.loc[str(j), 'Longitude']]))*((truck_vdc[(str(i), str(j))]))*(1/10) for j in VDCs if i != j]))
    
for i in vdcswithrails:
    model += (pulp.lpSum([(2000 + (3)*distance([locations_lp.loc[str(i), 'Latitude'], locations_lp.loc[str(i), 'Longitude']],[locations_lp.loc[str(j), 'Latitude'], locations_lp.loc[str(j), 'Longitude']]))*(rail_vdc[(str(i), str(j))])*(1/20) for j in vdcswithrails if i != j]))
    model += (pulp.lpSum([(2000 + (3)*distance([locations_lp.loc[str(i), 'Latitude'], locations_lp.loc[str(i), 'Longitude']],[locations_lp.loc[str(j), 'Latitude'], locations_lp.loc[str(j), 'Longitude']]))*(rail_dealer[(str(i), str(j))])*(1/20) for j in dealers_var]))
    

/Users/academic/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1349: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [ ]:
#Construct constraints

#model += pulp.lpSum([truck_vdc['3A', str(j)] for j in VDCs if str(j) != '3A']) + pulp.lpSum([truck_dealer['3A', str(k)] for k in dealers_3A])  == 237191
#model += pulp.lpSum([truck_dealer['3A', str(k)] + truck_dealer[str(j), str(k)] for k in dealers_3A for j in VDCs if str(j) != '3A']) == 237191

for k in new_index_3A:
    model += (arrive_by_truck[('3A', str(k))] == newdf_3A.loc[str(k), 'VIN'])
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == arrive_by_truck[('3A',str(k))])
    model += ((pulp.lpSum([truck_dealer[str(j), str(k)] for j in VDCs if j != '3A'])) + truck_dealer['3A', str(k)] + (pulp.lpSum([rail_dealer[str(j), str(k)] for j in vdcswithrails if j != '3A'])) + rail_dealer['3A', str(k)]  >= arrive_by_truck[('3A',str(k))])
    #model += ((pulp.lpSum([truck_vdc['3A', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != '3A'])) + (pulp.lpSum([rail_vdc['3A', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != '3A'])) == 0)
    


TypeError: must be real number, not LpAffineExpression

In [ ]:
#more constraints

for k in new_index_FF:
    model += (arrive_by_truck[('FF', str(k))] == newdf_FF.loc[str(k), 'VIN'])
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == arrive_by_truck[('FF',str(k))])
    model += ((pulp.lpSum([truck_dealer[str(j), str(k)] for j in VDCs if j != 'FF'])) + truck_dealer['FF', str(k)] + (pulp.lpSum([rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'FF'])) + rail_dealer['FF', str(k)]  >= arrive_by_truck[('FF',str(k))])
    #model += ((pulp.lpSum([truck_vdc['FF', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'FF'])) + (pulp.lpSum([rail_vdc['FF', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'FF'])) == 0)
    
        
for k in new_index_RO:
    model += (arrive_by_truck[('RO', str(k))] == newdf_RO.loc[str(k), 'VIN'])
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == arrive_by_truck[('RO',str(k))])
    model += ((pulp.lpSum([truck_dealer[str(j), str(k)] for j in VDCs if j != 'RO'])) + truck_dealer['RO', str(k)] + (pulp.lpSum([rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'RO'])) + rail_dealer['RO', str(k)]  >= arrive_by_truck[('RO',str(k))])
    #model += ((pulp.lpSum([truck_vdc['RO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'RO'])) + (pulp.lpSum([rail_vdc['RO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'RO'])) == 0)


for k in new_index_SO:
    model += (arrive_by_truck[('SO', str(k))] == newdf_SO.loc[str(k), 'VIN'])
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == arrive_by_truck[('SO',str(k))])
    model += ((pulp.lpSum([truck_dealer[str(j), str(k)] for j in VDCs if j != 'SO'])) + truck_dealer['SO', str(k)] + (pulp.lpSum([rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'SO'])) + rail_dealer['SO', str(k)]  >= arrive_by_truck[('SO',str(k))])
    #model += ((pulp.lpSum([truck_vdc['SO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'SO'])) + (pulp.lpSum([rail_vdc['SO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'SO'])) == 0)
    


In [ ]:
#more constraints



In [ ]:
#stored (inactive constraints)

"""model += ((pulp.lpSum([truck_vdc['RO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'RO'])) + (pulp.lpSum([rail_vdc['RO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'RO']))  == 0)
    model += ((pulp.lpSum([truck_vdc['SO', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'SO'])) + (pulp.lpSum([rail_vdc['SO', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'SO']))  == 0)
    model += ((pulp.lpSum([truck_vdc['FF', str(j)] - truck_dealer[str(j), str(k)] for j in VDCs if j != 'FF'])) + (pulp.lpSum([rail_vdc['FF', str(j)] - rail_dealer[str(j), str(k)] for j in vdcswithrails if j != 'FF']))  == 0)"""
for j in VDCs:
    if str(j) in ['3A','RO','SO','FF']:
        pass
    elif str(j) in vdcswithrails:
        model += (pulp.lpSum([truck_vdc[(str(i), str(j))] for i in VDCs if i != j]) + pulp.lpSum([rail_vdc[(str(i), str(j))] for i in vdcswithrails if i != j]) == (pulp.lpSum([truck_dealer[(str(j), str(k))] + rail_dealer[(str(j), str(k))] for k in dealers_var])))
    else:
        model += (pulp.lpSum([truck_vdc[(str(i), str(j))] for i in VDCs if i != j]) == (pulp.lpSum([truck_dealer[(str(j), str(k))] for k in dealers_var])))
        
        
        
    
#for k in combined_index:
    #model += ((pulp.lpSum([truck_dealer[str(i), str(k)] for i in VDCs]) + pulp.lpSum([rail_dealer[str(i), str(k)] for i in vdcswithrails])) == (pulp.lpSum([arrive_by_truck[str(i), str(k)] for i in ['3A', 'RO', 'SO', 'FF']])))
    #model += ((pulp.lpSum([truck_vdc[str(i), str(j)] - truck_dealer[str(j), str(k)] for i in ['RO','3A','SO','FF'] for j in VDCs if i != j])) + (pulp.lpSum([rail_vdc[str(i), str(j)] - rail_dealer[str(j), str(k)] for i in ['3A','RO','SO','FF'] for j in vdcswithrails if i != j]))  == 0)
    

In [ ]:
model.solve()

In [ ]:
pulp.LpStatus[model.status]

In [ ]:
print(pulp.value(model.objective))

In [ ]:
output = []
output2 = []
output3 = []
for i, j in truck_vdc:
    var_output = {
        'start': i,
        'mid': j,
        '# of cars': truck_vdc[(i, j)].varValue,
    }
    output.append(var_output)

for i, k in arrive_by_truck:
    var_output = {
        'start': i,
        'dealer': k,
        '# of cars': arrive_by_truck[(i, k)].varValue,
    }
    output3.append(var_output)

print(output)


In [100]:
start_truck_vdc = []
mid_truck_vdc = []
number_truck_vdc = []
for i, j in truck_vdc:
    start_truck_vdc.append(i)
    mid_truck_vdc.append(j)
    number_truck_vdc.append(truck_vdc[(i, j)].varValue)
    
start_truck_dealer = []
mid_truck_dealer = []
number_truck_dealer = []
for i, j in truck_dealer:
    start_truck_dealer.append(i)
    mid_truck_dealer.append(j)
    number_truck_dealer.append(truck_dealer[(i, j)].varValue)
    
start_rail_vdc = []
mid_rail_vdc = []
number_rail_vdc = []

for i, j in rail_vdc:
    start_rail_vdc.append(i)
    mid_rail_vdc.append(j)
    number_rail_vdc.append(rail_vdc[(i, j)].varValue)
    
start_rail_dealer = []
mid_rail_dealer = []
number_rail_dealer = []
for i, j in rail_dealer:
    start_rail_dealer.append(i)
    mid_rail_dealer.append(j)
    number_rail_dealer.append(rail_dealer[(i, j)].varValue)

In [102]:
df_truck_vdc = pd.DataFrame()
df_truck_vdc['Start of Path'] = start_truck_vdc
df_truck_vdc['End of Path'] = mid_truck_vdc
df_truck_vdc['Number of Cars on Path'] = number_truck_vdc
#df_truck_vdc['Distance'] = distances_truck_vdc
#df_truck_vdc['Cost'] = costs_truck_vdc

df_truck_dealer = pd.DataFrame()
df_truck_dealer['Start of Path'] = start_truck_dealer
df_truck_dealer['End of Path'] = mid_truck_dealer
df_truck_dealer['Number of Cars on Path'] = number_truck_dealer
#df_truck_dealer['Distance'] = distances_truck_dealer
#df_truck_dealer['Cost'] = costs_truck_dealer

df_rail_vdc = pd.DataFrame()
df_rail_vdc['Start of Path'] = start_rail_vdc
df_rail_vdc['End of Path'] = mid_rail_vdc
df_rail_vdc['Number of Cars on Path'] = number_rail_vdc
#df_rail_vdc['Distance'] = distances_rail_vdc
#df_rail_vdc['Cost'] = costs_rail_vdc

df_rail_dealer = pd.DataFrame()
df_rail_dealer['Start of Path'] = start_rail_dealer
df_rail_dealer['End of Path'] = mid_rail_dealer
df_rail_dealer['Number of Cars on Path'] = number_rail_dealer
#df_rail_dealer['Distance'] = distances_rail_dealer
#df_rail_dealer['Cost'] = costs_rail_dealer

In [96]:
df_truck_vdc = df_truck_vdc.dropna()
df_rail_vdc = df_rail_vdc.dropna()
df_truck_dealer = df_truck_dealer.dropna()
df_rail_dealer = df_rail_dealer.dropna()

In [87]:
df_truck_vdc.to_csv('truck_vdc_routingdetails.csv')
df_truck_dealer.to_csv('truck_dealer_routingdetail.csv')
df_rail_vdc.to_csv('rail_vdc_routingdetail.csv')
df_rail_dealer.to_csv('rail_dealer_routingdetail.csv')

In [ ]:
#check
#print(vins_3A)
for k in new_index_3A:
    print(type(newdf_3A.loc[str(k), 'VIN']))

In [ ]:
15//10

In [9]:
dffinal1 = pd.read_csv('truck_dealer_routingdetail2.csv',names=['index','start','end','number'],header=0,usecols=[0,1,2,3])
dffinal2 = pd.read_csv('rail_dealer_routingdetail2.csv',names=['index','start','end','number'],header=0,usecols=[0,1,2,3])


In [12]:
distances_truck_vdc = []
distances_rail_vdc = []
distances_truck_dealer = []
distances_rail_dealer = []

startingtruck = dffinal1['start'].tolist()
midtruck = dffinal1['end'].tolist()
numbertruck = dffinal1['number'].tolist()

startingrail = dffinal2['start'].tolist()
midrail = dffinal2['end'].tolist()
numberrail = dffinal2['number'].tolist()
    
for i in range(len(startingtruck)):
    temp1 = distance([locations_lp.loc[str(startingtruck[i]), 'Latitude'], locations_lp.loc[str(startingtruck[i]), 'Longitude']],[locations_lp.loc[str(midtruck[i]), 'Latitude'], locations_lp.loc[str(midtruck[i]), 'Longitude']])
    distances_truck_dealer.append(temp1)

for i in range(len(startingrail)):
    temp1 = distance([locations_lp.loc[str(startingrail[i]), 'Latitude'], locations_lp.loc[str(startingrail[i]), 'Longitude']],[locations_lp.loc[str(midrail[i]), 'Latitude'], locations_lp.loc[str(midrail[i]), 'Longitude']])
    distances_rail_dealer.append(temp1)
    

print(distances_truck_dealer[:100])

[1755.5454123034785, 1247.5439248774774, 1785.045437166107, 1697.6736017903493, 941.6640554511861, 625.5882320502417, 945.5698013523294, 1618.4903195597224, 1022.9558642936422, 1567.2150737809961, 1761.2013931985186, 858.7749209151733, 2216.352585026631, 193.77038165130338, 1411.6250312569155, 1327.9833946518268, 550.934466983487, 350.8125434605505, 1298.2899090238216, 1722.6574972569717, 1218.3619204770735, 995.0001485067226, 561.2471168672694, 1419.1489827428632, 524.4282577958198, 1463.188669336892, 797.282358862904, 787.5370860576589, 438.7737589064111, 877.7549075598819, 776.8543394642039, 1366.049212482919, 1274.4524412327005, 760.7016141318009, 1217.6364929880244, 833.9748781783243, 884.3587588245299, 976.5048406709125, 947.2677672221392, 936.006322881239, 640.162158670063, 1708.82101282975, 1734.074769820091, 1532.3257375406577, 1203.8451996042575, 1199.681870581671, 1320.3432560021681, 328.3400826758245, 133.0945606677574, 1066.9116760365962, 1533.6819595109382, 1482.728104467

In [20]:
costs_truck_vdc = []
costs_truck_dealer = []
costs_rail_vdc = []
costs_rail_dealer = []


    
for i in range(len(distances_truck_dealer)):
    if numbertruck[i] == 0:
        costs_truck_dealer.append(0)
    else:
        temp = (numbertruck[i]//10 + 1)*(4*distances_truck_dealer[i] + 200)
        costs_truck_dealer.append(temp)
    

    
for i in range(len(distances_rail_dealer)):
    if numberrail[i] == 0:
        costs_rail_dealer.append(0)
    else:
        temp = (numberrail[i]//10 + 1)*(4*distances_rail_dealer[i] + 200)
        costs_rail_dealer.append(temp)
    
print(costs_truck_dealer[:100])

[7222.181649213914, 0, 0, 20972.08322148419, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99390.50212547026, 0, 14422.427207603689, 0, 0, 0, 0, 0, 12224.942337345386, 0, 6893.139093549838, 0, 0, 0, 0, 0, 0, 0, 10595.619529861604, 35670.871021799234, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33142.892481058516, 28283.008462317244, 0, 0, 0, 11211.788140524068, 0, 0, 0, 59131.87071177224, 0, 0, 0, 0, 0, 0, 18799.980381918922, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6734.259227643834, 0, 0, 0, 0, 0, 0, 0, 6224.701437540112, 0, 0, 0, 0, 0, 0]


In [22]:
suma = sum(costs_truck_dealer) + sum(costs_rail_dealer)
print(suma)
#print(sum(costs_truck_dealer)
#print(sum(costs_rail_vdc)
#print(sum(costs_rail_dealer)

dffinal1['Distance'] = distances_truck_dealer
dffinal1['Cost'] = costs_truck_dealer

dffinal2['Distance'] = distances_rail_dealer
dffinal2['Cost'] = costs_rail_dealer

    
    

258340674.55925


In [23]:
dffinal1.to_csv('truck_routingdetail_optimized.csv')
dffinal2.to_csv('rail_routingdetail_optimized.csv')

In [ ]:
for i in range(len(distances_truck_vdc)):
    temp = (number_truck_vdc[i]//10 + 1)*(4*distances_truck_vdc[i] + 200)
    costs_truck_vdc.append(temp)
    
    
for i in range(len(distances_rail_vdc)):
    temp = (number_rail_vdc[i]//10 + 1)*(4*distances_rail_vdc[i] + 200)
    costs_rail_vdc.append(temp)